In [2]:
pip install mlbox

     |████████████████████████████████| 20.2 MB 1.2 MB/s 
     |████████████████████████████████| 13.0 MB 58.8 MB/s 
     |████████████████████████████████| 1.9 MB 52.2 MB/s 
     |████████████████████████████████| 10.4 MB 44.7 MB/s 
     |████████████████████████████████| 294 kB 66.8 MB/s 
     |████████████████████████████████| 7.0 MB 52.1 MB/s 
     |████████████████████████████████| 86.3 MB 64 kB/s 
     |████████████████████████████████| 1.2 MB 43.6 MB/s 
     |████████████████████████████████| 4.3 MB 42.0 MB/s 
     |████████████████████████████████| 103 kB 72.2 MB/s 
     |████████████████████████████████| 1.7 MB 51.3 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 449 kB 55.8 MB/s 
     |████████████████████████████████| 3.8 MB 46.9 MB/s 
  Created wheel for mlbox: filename=mlbox-0.8.5-py3-none-any.whl size=43755 sha256=3d8ead4bacdfd787ed408df03516a2d54444222079f4c253e18d2c43183bfa4c
  Stored in directory: /root/.cache/pip/w

In [3]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

In [6]:
path = ["/content/drive/MyDrive/Colab Notebooks/autokeras/Dados/Churn_treino.csv","/content/drive/MyDrive/Colab Notebooks/autokeras/Dados/Churn_teste.csv"]

In [7]:
imp = Reader(sep = ";")
data = imp.train_test_split(path, "Exited")


reading csv : Churn_treino.csv ...
cleaning data ...
CPU time: 6.922170639038086 seconds

reading csv : Churn_teste.csv ...
cleaning data ...
CPU time: 0.48473548889160156 seconds

> Number of common features : 10

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 2
> Number of numerical features: 8
> Number of training samples : 10000
> Number of test samples : 2474

> You have no missing values on train set...

> Task : classification
0.0    7963
1.0    2037
Name: Exited, dtype: int64

encoding target ...


In [8]:
# Remove drift

rdrift = Drift_thresholder()
data = rdrift.fit_transform(data)


computing drifts ...
CPU time: 5.827667951583862 seconds

> Top 10 drifts

('EstimatedSalary', 0.04372004850444622)
('Age', 0.04050735650767989)
('CreditScore', 0.031266127728375004)
('Tenure', 0.017684640258690543)
('HasCrCard', 0.01761350040420373)
('NumOfProducts', 0.016328455941794662)
('Balance', 0.0100167340339532)
('Gender', 0.009821907841552235)
('Geography', 0.0055738884397738)
('IsActiveMember', 0.004916814874696751)

> Deleted variables : []
> Drift coefficients dumped into directory : save


In [9]:
# optimize object
optimize = Optimiser()

/usr/local/lib/python3.7/dist-packages/mlbox/optimisation/optimiser.py:74: UserWarning: Optimiser will save all your fitted models into directory 'save/joblib'. Please clear it regularly.
  +str(self.to_path)+"/joblib'. Please clear it regularly.")


In [10]:
#Optimodr parameters, of the dictionary type
espace = {
     'fs__strategy':{"search":"choice","space":["variance","rf_feature_importance"]},
     'est__colsample_bytree':{"search":"uniform", "space":[0.3,0.7]}
}

In [11]:
#Creation of the actual model
# max_evals is the interactions, default is 40
# I collected 15 epochs just for testing and studying.
model = optimize.optimise(espace,data,max_evals=15)

##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'label_encoding'}
>>> FEATURE SELECTOR :{'strategy': 'variance', 'threshold': 0.3}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'colsample_bytree': 0.6265931446441935, 'boosting_type': 'gbdt', 'class_weight': None, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 0.9, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'nthread': -1, 'seed': 0}
MEAN SCORE : neg_log_loss = -0.4044676216181195
VARIANCE : 0.0015177058509187602 (fold 1 = -0.40598532746903826, fold 2 = -0.40294991576720074)
CPU time: 11.584

In [12]:
#forecast
forecast = Predictor()
forecast.fit_predict(model, data)


fitting the pipeline ...
CPU time: 9.283700466156006 seconds

predicting ...
CPU time: 0.08116960525512695 seconds

> Overview on predictions : 

        0.0       1.0  Exited_predicted
0  0.706120  0.293880                 0
1  0.980511  0.019489                 0
2  0.936784  0.063216                 0
3  0.207324  0.792676                 1
4  0.370915  0.629085                 1
5  0.952282  0.047718                 0
6  0.928365  0.071635                 0
7  0.635891  0.364109                 0
8  0.353668  0.646332                 1
9  0.760367  0.239633                 0

dumping predictions into directory : save ...


/usr/local/lib/python3.7/dist-packages/mlbox/prediction/predictor.py:392: UserWarning: Unable to get feature importances !
  warnings.warn("Unable to get feature importances !")
